Nome: Vinicius Freitas Schiavinato Olzon


Matrícula: 20210026803


- O objetivo deste exercício é treinar e avaliar um classificador Naive Bayes para
uma tarefa de análise de sentimentos.
- O texto utilizado no treinamento e no teste deve ser pré-processado para incluir
o prefixo “NAO_” de acordo com a seguinte regra:
- O prefixo “NAO_” deve ser adicionado a cada palavra após um
token de negação (não, nem, nunca, jamais, tampouco) até a próxima
pontuação. São consideradas pontuações: . , ? ! ;
- O treino deve ser realizado com 80% dos dados e o teste com 20%. Você deve
calcular a acurácia do classificador aplicado aos dados de teste e comparar com
o resultado do Naive Bayes sem a utilização do prefixo “NAO_”.

In [95]:
import pandas as pd
from unidecode import unidecode

# Carrega os dados

In [96]:
df = pd.read_csv('B2W-Reviews01.csv')

/tmp/ipykernel_18700/1153590322.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('B2W-Reviews01.csv')


### Pré-processamento das avaliações do dataset

In [97]:
df = df.dropna(subset=['review_text'])
df = df.reset_index(drop=True)
df_processado = df.copy()

O prefixo “NAO_” deve ser adicionado a cada palavra após um
token de negação (não, nem, nunca, jamais, tampouco) até a próxima
pontuação. São consideradas pontuações: . , ? ! ;

In [98]:
def adiciona_prefixo(text):
    prefixo = ' NAO_ '
    pontuacoes = ['.', ',', '!', '?']
    tokens = ['nao', 'nem', 'nunca', 'jamais', 'tampouco']

    text = unidecode(text).lower()
    i = 0
    ativado = False
    while i < len(text.split()):
        if ativado:
            # insere o prefixo ' NAO_ ' em frente a palavra (depois ele pula um índice/palavra -> i+=1)
            texto_antes = ' '.join(str(word) for word in text.split()[:i])
            texto_depois = ' '.join(str(word) for word in text.split()[i:])
            # unidecode retira os acentos e o .lower() deixa tudo minúsculo para padronizar o texto
            text = texto_antes + prefixo + texto_depois
            
            i+=1

        # se o algoritmo encontra algumas das pontuações da lista 'pontuacoes', ele para de inserir o prefixo ' NAO_ ' em frente as palavras
        if text.split()[i] in pontuacoes or text.split()[i][-1] in pontuacoes:
            ativado = False

        # se o algoritmo encontra algumas das palavras da lista 'tokens', ele passa a inserir o prefixo ' NAO_ ' em frente as palavras
        if text.split()[i] in tokens:
            ativado = True

        i+=1

    return text

In [99]:
reviews_processados = df_processado['review_text']
len(reviews_processados)
for i in range(len(reviews_processados)):
    reviews_processados[i] = adiciona_prefixo(reviews_processados[i])

In [101]:
df['review_text'][11]

'Produto maravilhoso! Não é barulhento, fácil manuseio, tranquilo de montar e desmontar, entrega antes do prazo mesmo se tratando de época do Natal. Americanas de parabéns e o produto muito bom. Recomendo com força.'

In [103]:
df_processado['review_text'][11]

'produto maravilhoso! nao NAO_ e NAO_ barulhento, facil manuseio, tranquilo de montar e desmontar, entrega antes do prazo mesmo se tratando de epoca do natal. americanas de parabens e o produto muito bom. recomendo com forca.'

### reviews com menos de 3 estrelas são tratadas como negativas (0) e o resto será tratado como positivo (1)

In [104]:
df['label'] = df['overall_rating'].apply(lambda x: 0 if x <=3 else 1)
df_processado['label'] = df_processado['overall_rating'].apply(lambda x: 0 if x <=3 else 1)


# Naive Bayes com o dataset original

In [105]:
X = df['review_text']
y = df['label']

### Dividindo os dados em dados de treino e teste

In [106]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [108]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
model = classifier.fit(X_train_vec, y_train)

In [109]:
model.predict(X_test_vec)

array([1, 1, 0, ..., 1, 1, 0])

In [110]:
from sklearn.metrics import accuracy_score

acuracia = accuracy_score(y_test, model.predict(X_test_vec))

In [111]:
my_input = 'ótimo produto, não foi muito barato'
my_input = tfidf.transform([my_input])
print(model.predict(my_input))

[1]


### Naive Bayes com o dataset processado com os prefixos

In [112]:
X = df_processado['review_text']
y = df_processado['label']

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [114]:
tfidf = TfidfVectorizer()
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [115]:
classifier = MultinomialNB()
new_model = classifier.fit(X_train_vec, y_train)

In [116]:
new_model.predict(X_test_vec)

array([1, 0, 1, ..., 1, 1, 0])

In [117]:
acuracia_prefixo = accuracy_score(y_test, new_model.predict(X_test_vec))

In [118]:
my_input = 'ótimo produto, não odiei'
my_input = tfidf.transform([my_input])
print(new_model.predict(my_input))

[0]


In [119]:
my_input = 'um jeito muito bom de gastar muito dinheiro, gostei bem pouco'
my_input = tfidf.transform([my_input])
print(new_model.predict(my_input))

[1]


### Ao analisar a acurácia de ambos os modelos, pode ser visto abaixo que não há muita diferença com a adição dos prefixos no textos de avaliações

In [120]:
acuracia_prefixo

0.853718048024787

In [121]:
acuracia

0.8568938807126258